In [1]:
# créer un nouvel env conda à partir du terminal
# conda create --name pathrag python=3.10
# installer ollama: https://www.ollama.com/download

# installer les dépendences
#%pip install -r requirements.txt
# intsaller le modème d'OllamaEmbeddings
!ollama pull nomic-embed-text
# créer une clé api sur openrouter pour utiliser des llm gratuitement

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90: 100% ▕██████████████████▏ 274 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling ce4a164fc046: 100% ▕██████████████████▏   17 B                         
pulling 31df23ea7daa: 100% ▕██████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
from openai import OpenAI, AsyncOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.schema.document import Document
from pathrag_retriever import create_graphdb, load_existing_graphdb, load_knowledgeGraph_vis
import time


/home/chougar/miniconda3/envs/pathrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Base vectorielle
Création de la base vect. avec le PP Mahakam (20 premières pages, ligne 45 `docs[:20]`)

Relancer la cellule à chaque ouverture pour utiliser le rag vectoriel

In [3]:
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.retrievers import TFIDFRetriever
from langchain_unstructured import UnstructuredLoader

#========= choix du modèle d'embedding
"""
    Le modèle choisi impacte la qualité du retriever, mais aussi le temps de traitement
    Si le déploiement est prévu sur une VM limitée, un modèle plus petit est nécessaire
    Explorer les comparatifs: https://huggingface.co/spaces/mteb/leaderboard

"""
# Utiliser OllamaEmbeddings avec le modèle local "nomic-embed-text"
embeddings = OllamaEmbeddings(model="nomic-embed-text")



# chargement et fragmentation du doc
filename="PU_P01_PP01.docx"
doc_name_hybrid="PP mahakam light" # nom de doc significatif


# loader = UnstructuredFileLoader(filename)
loader = UnstructuredLoader(filename)

docx_docs = loader.load()
print(f"Loaded {len(docx_docs)} documents from {filename}")


#======== choix des paramètres de fragmentation
"""
    la taille du chunck_size est très important dans l'accès à une info précise
    une plus petite taille permet de cibler de courts passages contenant l'info nécessaire à des réponses précises:
        * lieu du projet
        * dates du projet
        * budget ...    
    l'envoi de passages plus courts au llm évite une dispertion de son attention
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len
)

docs = text_splitter.split_documents(docx_docs)

# Filter out complex metadata (e.g., lists, dicts)
docs = [Document(doc.page_content) for doc in docs[:20]]

print(len(docs))

# Conversion des docs en embeddings 
chroma_db = Chroma.from_documents(
    docs,
    embedding=embeddings,
    persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
    collection_name=doc_name_hybrid.replace(" ","_")
)

retriever=chroma_db.as_retriever()





# ...existing code...
all_docs = chroma_db.get()
print(len(all_docs['documents']))  # This will print the total number of docs stored
# ...existing code...

Loaded 533 documents from PU_P01_PP01.docx
20


INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
ERROR: Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR: Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
INFO: HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
ERROR: Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


60


#### Création ou chargement d'un graphe existant:
Saisir l'action désirée dans le prompt qui s'affiche en haut du notebook, suivre instructions

Par défaut prise en compte des 20 premières pages pour l'exemple, modifier ligne 11 `for doc in docx_docs[:20]:`

Si vous voulez charger un graphe déjà crée, retrouver son nom dans le fichier `graphrag_hashes.json`, attribut `Nom du doc`

Si vous voulez modifier le LLM utilisé pour la création du graphe ou sa lecture, allez dans `pathrag_retriever.py`, ligne 34 et 35, et prenez un modèle valide sur openrouter (attention à prendre un modèle qui supporte les `structured_outputs`, à filtrer à droite dans la liste des `Supported parameters`)

**Important**: renseigner votre clé api openrouter sur le script `pathrag_retriever.py`, ligne 42

In [4]:
# appliquer nest_asyncio uniquement sur notebook pour corriger l'erreur de loop event
import nest_asyncio
nest_asyncio.apply()

# remetre à plat le text
filename="PU_P01_PP01.docx"
loader = UnstructuredLoader(filename)

docx_docs = loader.load()
text=""
for doc in docx_docs[:20]:
    text+=doc.page_content


r=input("Saisir 'C' pour créer un nouveau graphe, 'L' pour charger un graphe existant")

# créer un nouveau graphe
messages=None
if r=='C':
    doc_name_graph=input('Saisir un nom unique pour votre graphe')
    print(f"Le nom de votre graphe est {doc_name_graph}")
    messages= create_graphdb(
        text=text, 
        doc_name=doc_name_graph, # il faut donner un nom unique permettant d'identifier et charger le graph les prochaines fois
    )
# charger un graphe existant
elif r=='L':
    doc_name_graph=input('Saisir le nom du graphe à charger')
    print(f"Le nom de votre graphe est {doc_name_graph}")

    messages=load_existing_graphdb(doc_name_graph)
else:
    print('Option invalide')



if messages:
    pipeline_args={}
    for feedback in messages:
        if isinstance(feedback, str):
            print(feedback)
        elif isinstance(feedback, dict):
            pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]=feedback["pipeline_args"]
            


INFO:PathRAG:Logger initialized for working directory: /home/chougar/Documents/GitHub/Projet-portail-immo/docs-to-rag share/storage/graph_stores/6eb62bb393f33532a59e24b774045344e61fdc93747de585e719bc100467d2a0
INFO:PathRAG:Load KV llm_response_cache with 0 data
INFO:PathRAG:Load KV full_docs with 1 data
INFO:PathRAG:Load KV text_chunks with 1 data
INFO:PathRAG:Loaded graph from /home/chougar/Documents/GitHub/Projet-portail-immo/docs-to-rag share/storage/graph_stores/6eb62bb393f33532a59e24b774045344e61fdc93747de585e719bc100467d2a0/graph_chunk_entity_relation.graphml with 33 nodes, 16 edges
INFO:nano-vectordb:Load (32, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Documents/GitHub/Projet-portail-immo/docs-to-rag share/storage/graph_stores/6eb62bb393f33532a59e24b774045344e61fdc93747de585e719bc100467d2a0/vdb_entities.json'} 32 data
INFO:nano-vectordb:Load (16, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'c

Le nom de votre graphe est PP mahakam light

        ----------------
        #### Graph RAG retriever
        Chargement de la base Graph RAG
    
**✅ Graph RAG chargé**


In [5]:
from PathRAG import QueryParam
import asyncio

def stream_pathRAG_response(stream_resp):
    async def stream_response():        
        # Process the async generator
        async for chunk in stream_resp:
            print(chunk or "", end="")


    # Run in Streamlit's existing event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(stream_response())

# question="résume ce texte dans sa langue source"
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"

resp=pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]["rag"].query(query= question, param=QueryParam(mode="hybrid", stream=True))

stream_pathRAG_response(resp)

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:PathRAG:kw_prompt result:
INFO:PathRAG:```json
{
  "high_level_keywords": ["Analyse textuelle", "Thèmes principaux", "Questions de compréhension"],
  "low_level_keywords": ["Identification des thèmes", "Formulation de questions", "Lecture analytique", "Compréhension de texte"]
}
```
INFO:PathRAG:Local query uses 32 entites, 15 relations, 1 text units
INFO:PathRAG:Global query uses 20 entites, 16 relations, 1 text units
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


response all ready


## Principaux thèmes et questions potentielles du texte

Le texte présente un projet de restauration de mangroves dans l’est de Kalimantan, en Indonésie, et aborde plusieurs thèmes clés interconnectés.  Voici une analyse des principaux thèmes et des questions qu’ils soulèvent :

**1. Dégradation de l'environnement et restauration des écosystèmes :**

*   **Description :** Un thème central est la dégradation des écosystèmes de mangroves dans le delta du Mahakam et la baie d'Adang en raison de l'exploitation excessive des terres pour l'aquaculture. Le projet se concentre sur la restauration de ces mangroves dégradées.
*   **Questions potentielles :**
    *   Quels sont les impacts spécifiques de la dégradation des mangroves sur l'environnement local et les communautés ?
    *   Quelles sont les techniques de restauration des mangroves qui seront utilisées dans le projet ?
    *   Existe-t-il des conflits d'intérêts entre les activités d'aquaculture et la restauration

Renseigner votre clé d'api sur la ligne 20

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.schema.document import Document
from openai import OpenAI, AsyncOpenAI
import asyncio
import json
import re

class RAG_hybrid():
    def __init__(self, model):
        self.model=model
        self.retrieved_docs=[]
        self.semantic_retriever_topK=10
        self.sparse_retriever_topK=10
        self.history=[]
        self.llm_client = AsyncOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key="",
        )
        self.reranker_llm="mistralai/mistral-small-3.1-24b-instruct:free"
        self.reranker_score_thresh=5
        self.reranked_doc=[]

    def semanticRetriever(self):
        # 1. Semantic Retriever (Chroma + OllamaEmbeddings)
        embeddings = OllamaEmbeddings(model="nomic-embed-text")
        chroma_db = Chroma(
            persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
            collection_name=doc_name_hybrid.replace(" ","_"),
            embedding_function=embeddings
        )

        semantic_retriever=chroma_db.as_retriever(search_type="mmr", k=self.semantic_retriever_topK)

        self.chroma_db=chroma_db
        self.semantic_retriever=semantic_retriever
    
    def sparseRetriever(self):
        # 2. Sparse Retriever (TF-IDF)

        # Récupérer TOUS les documents depuis Chroma
        all_data = self.chroma_db.get(include=["documents", "metadatas"])

        # Convertir en liste de `Document` objects pour LangChain
        docs = [
            Document(page_content=text, metadata=meta or {})  # <-- Si meta est None, on met {}
            for text, meta in zip(all_data["documents"], all_data["metadatas"])
        ]

        # Créer le retriever TF-IDF
        sparse_retriever = TFIDFRetriever.from_documents(
            documents=docs,
            k=self.sparse_retriever_topK,
            tfidf_params={"min_df": 1, "ngram_range": (1, 2)}
        )

        self.sparse_retriever= sparse_retriever
    
    def ensembleRetriever(self):
        # 3. Ensemble Retriever (Semantic + Sparse)
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.semantic_retriever, self.sparse_retriever],
            weights=[0.5, 0.5]
        )

        self.ensemble_retriever=ensemble_retriever

    async def reranker(self, results, query):


        async def llm_eval(doc, query):
            system_prompt="""
                You're an expert assistant in reranking documents against a question.
                Your role is to compare the question with a document and give a score from 0 to 10, where:
                0=document out of context, unable to answer the question
                10=highly relevant document, able to answer the question
                                
                The expected final output is the score in json format
                Example:
                ```json{"score": 5}```
                
                Always end your answer with this format                
            """            
            response = await self.llm_client.chat.completions.create(
                model=self.reranker_llm,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"La question est: {query}\n Le document à évaluer est le suivant\n: {doc}" }
                ],
                temperature=0,
            )
            # Post-process to extract only the JSON part if extra text is present
            content = response.choices[0].message.content
            # Try to extract the JSON block if the model adds extra text
            match = re.search(r"\{.*?\}", content, re.DOTALL)
            if match:
                content = match.group(0)

            # extract score
            score=None
            try:
                score=content.replace("```json", "").replace("```", "")
                
                score= json.loads(score)
                score=score["score"]
            except Exception as e:
                print(e)                
            
            return {"content": doc, "score": score}


        tasks=[llm_eval(doc.page_content, query) for doc in results]
        scored_docs= await asyncio.gather(*tasks)
        i=1

        for doc in scored_docs:
          
            print(f'chunk {i} score: {doc["score"]}')
            i+=1

        filtred_docs=[d for d in scored_docs if d["score"]>=self.reranker_score_thresh]
        # print(f"scored docs; \n{scored_docs}")
        self.reranked_doc=filtred_docs

        return filtred_docs

    async def ask_llm(self, query):
        # 5. Final processing step with an LLM (e.g., OpenAI via OpenRouter)

        # init retrievers
        self.semanticRetriever()
        self.sparseRetriever()
        self.ensembleRetriever()

        # retrieve relevant docs
        results = self.ensemble_retriever.get_relevant_documents(query)
        print(f"Nb of retrieved docs: {len(results)}")

        # rerank
        scored_results=await self.reranker(results, query)
        
        # Concatenate retrieved documents for context
        context = "\n".join([f"Fragment: \n{doc['content']}\n" for doc in scored_results])

        print(f"Context lenght: {len(context.split(' '))} words")
        llm_prompt = f"""
            Answer the question based **only** on the provided context.  

            - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
            - If the context lacks relevant information, respond with: "I don't know."  

            ### **Context:**  
            {context}  

            ### **Question:**  
            {query}  

            ### **Answer:**  
            Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
        """

        llm_completion = await self.llm_client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in document Q/A and document synthesis"},
                {"role": "user", "content": llm_prompt}
            ],
            temperature=0.2,
            stream=True
        )

        final_answer = ""
        print("Réponse:\n=========")
        async for chunk in llm_completion:
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                final_answer += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="", flush=True)
        
        self.history+=[
            {"role": "user", 'content': query},
            {"role": "assistant", "content": final_answer}
        ]
        
        return final_answer



In [7]:
rag_hybrid=RAG_hybrid(model="mistralai/mistral-small-3.2-24b-instruct:free")
# 4. Ask a question
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"
results = await rag_hybrid.ask_llm(question)

ERROR: Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR: Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Nb of retrieved docs: 12


INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: P

chunk 1 score: 3
chunk 2 score: 7
chunk 3 score: 0
chunk 4 score: 3
chunk 5 score: 5
chunk 6 score: 0
chunk 7 score: 3
chunk 8 score: 6
chunk 9 score: 9
chunk 10 score: 4
chunk 11 score: 3
chunk 12 score: 2
Context lenght: 181 words


INFO: HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Réponse:
Les principaux thèmes abordés dans le texte sont :

1. **Conservation des écosystèmes** : Le texte mentionne les efforts de conservation menés par différentes parties et le gouvernement pour protéger l'écosystème de la Delta Mahakam et de la baie d'Adang.

2. **Déforestation et dégradation des mangroves** : Il est question de la dégradation des mangroves due à la conversion des terres en zones d'aquaculture, avec une mention spécifique de 47,5 % de l'écosystème mangrove dégradé en 2017.

3. **Espèces menacées** : Le texte souligne la présence d'espèces en danger critique d'extinction, comme le nasique (proboscis monkey), endémique à l'île de Bornéo.

4. **Changement climatique** : Les mangroves sont décrites comme un écosystème clé pour la séquestration du carbone et la lutte contre le changement climatique.

5. **Gestion des terres** : Il est fait référence à la propriété des terres dans la Delta Mahakam, qui est une terre gouvernementale désignée comme une forêt de productio